This notebook contains the steps taken in the paper regarding:
 - Semi-Supervised training for HS detection.
 - Semi-Supervised training for CS detection.
 - Topic Modeling of HS comments.

All parameters for the different methods can be finetuned as desired.

# Libraries and Mock Data

## Libraries

In [ ]:
# Standard Libraries
import os
import json
import re
import string
import pickle
import random

# Numerical and Data Libraries
import numpy as np
import pandas as pd
from tqdm import tqdm

# NLP Libraries
import spacy
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from textblob import TextBlob
from sentence_transformers import SentenceTransformer

# Machine Learning and Deep Learning
import torch
from torch.utils.data import DataLoader, Subset, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.metrics import classification_report, accuracy_score

# Transformers and Hugging Face
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    BertTokenizer, BertForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from llama_cpp import Llama


# Topic Modeling and Clustering
import hdbscan
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, LlamaCPP, MaximalMarginalRelevance, PartOfSpeech

# Gensim (Topic Coherence)
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

# Datasets
from datasets import Dataset, load_dataset, concatenate_datasets

# Initial Setup
nltk.download('wordnet')
nlp = spacy.load("en_core_web_sm")

# Environment Variables
TF_ENABLE_ONEDNN_OPTS = 0


## Mock Data
The following mock data has been generated with ChatGPT.

Mock Data Labeled Datasets HS

In [2]:
def generate_mock_hate_labeled_data():
    texts = [
        "This is a terrible comment.",  # hate
        "You're the worst person ever.",  # hate
        "I can't stand your opinions.",  # hate
        "This is hateful and unacceptable.",  # hate
        "I will report this offensive post.",  # hate
        "Stop posting such hateful things.",  # hate
        "This content is deeply offensive.",  # hate
        "This is pure hate speech.",  # hate
        "You should be banned for this.",  # hate
        "This comment is abusive.",  # hate
        "Great job on the project!",  # non-hate
        "I agree with this viewpoint.",  # non-hate
        "This is a very informative post.",  # non-hate
        "Well done on explaining this topic.",  # non-hate
        "I support this initiative fully.",  # non-hate
        "This is a good contribution to the discussion.",  # non-hate
        "Thank you for sharing this.",  # non-hate
        "This is a positive and helpful comment.",  # non-hate
        "Excellent work, keep it up!",  # non-hate
        "I appreciate your insight on this matter."  # non-hate
    ]
    labels = [1] * 10 + [0] * 10  # First 10 are hate (1), last 10 are non-hate (0)
    return pd.DataFrame({'text': texts, 'label': labels})

mock_data_hate_labeled = generate_mock_hate_labeled_data()

Mock Data Labeled Datasets CS

In [3]:
data = {
    'dialogue_id': [
        '1001', '1001', '1002', '1002', '1003',
        '1003', '1004', '1004', '1005', '1005',
        '1006', '1006', '1007', '1007', '1008',
        '1008', '1009', '1009', '1010', '1010',
        '1011', '1011', '1012', '1012', '1013',
        '1013', '1014', '1014', '1015', '1015',
        '1016', '1016', '1017', '1017', '1018',
        '1018', '1019', '1019', '1020', '1020'
    ],
    'text': [
        "You're so stupid, I can't believe anyone listens to you.",  # HS
        "It's important to stay respectful even when we disagree.",  # CN
        "Why do people like you always spread misinformation?",  # HS
        "Everyone has different opinions, and that's okay.",  # CN
        "People like you are ruining the discussion with your hatred.",  # HS
        "Let's focus on constructive dialogue rather than insults.",  # CN
        "I find your comments deeply offensive and ignorant.",  # HS
        "We should be more empathetic and try to understand each other's perspectives.",  # CN
        "This kind of hate speech should not be tolerated.",  # HS
        "Respectful conversation is key to resolving conflicts.",  # CN
        "Your views are so outdated and discriminatory.",  # HS
        "We can have a meaningful debate without resorting to personal attacks.",  # CN
        "You only spread negativity and hate.",  # HS
        "Constructive criticism is a way to improve, not to insult.",  # CN
        "This rhetoric only fosters division and hatred.",  # HS
        "Let's engage in discussions that promote mutual respect.",  # CN
        "Hate speech has no place in our society.",  # HS
        "Understanding and compassion can bridge gaps between us.",  # CN
        "Your comment is a perfect example of toxic behavior.",  # HS
        "We should strive for a more inclusive and respectful dialogue.",  # CN
        "I can't believe how ignorant some people can be.",  # HS
        "Engaging in respectful dialogue can help us learn from each other.",  # CN
        "Your argument is completely unfounded and offensive.",  # HS
        "Let's focus on finding common ground rather than attacking each other.",  # CN
        "Your language is very hurtful and damaging.",  # HS
        "Promoting understanding is essential for constructive discussions.",  # CN
        "This kind of dialogue only escalates conflict.",  # HS
        "We should aim for conversations that are both respectful and productive.",  # CN
        "Your statements are harmful and unnecessary.",  # HS
        "Open-mindedness and respect are crucial for meaningful exchanges.",  # CN
        "This kind of negativity is detrimental to productive conversation.",  # HS
        "Constructive dialogue involves listening and understanding.",  # CN
        "Your comments only serve to create further division.",  # HS
        "Let's work towards resolving our differences with mutual respect.",  # CN
        "This type of rhetoric is unacceptable in any discussion.",  # HS
        "We should always strive to communicate with empathy and respect.",  # CN
        "Your behavior is completely unacceptable and damaging.",  # HS
        "Respectful discourse is the foundation of any healthy discussion.",  # CN
        "I do not like you.", # HS
        "You should not answer like this." # CN
    ],
    'turn_id': [
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1
    ],
    'label': [
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1
    ]
}

mock_dialogues_df = pd.DataFrame(data)

Mock Data TikTok Comments

In [4]:
mock_data_comments = {
    "cid": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    "text": [
        # Non-hate comments
        "I totally agree with this post!",
        "Great job on explaining the topic.",
        "This is such a well-written article, thanks for sharing!",
        "Your insights are very helpful, I appreciate the effort.",
        "This is an interesting perspective, and I learned something new.",
        "I'm really impressed by the clarity of your explanation.",
        "I couldn't have said it better myself. Well done!",
        "This was a very informative read, thanks for posting it!",
        "I really appreciate the thoughtfulness of your comments.",
        "Such a positive contribution to the discussion!",
        
        # Hateful comments
        "You're such an idiot for believing this garbage!",
        "This is disgusting! People like you make me sick!",
        "I can't stand how stupid you are, do us all a favor and stop talking.",
        "Only morons would agree with this nonsense, you must be brain-dead!",
        "You're a worthless piece of trash, no one cares about your opinion!",
        "What a waste of time, you're nothing but a pathetic loser!",
        "It's hard to believe anyone is as dumb as you. Just shut up already.",
        "You're the reason why the world is so messed up. Go crawl under a rock!",
        "If ignorance was a crime, you'd be serving life. This is just sad.",
        "Do everyone a favor and disappear, your existence is pointless!"
    ]
}

mock_comments_df = pd.DataFrame(mock_data_comments)

Mock Data TikTok Conversations

In [5]:

data = {
    'dialogue_id': [
        '3001', '3001', '3002', '3002', '3003',
        '3003', '3004', '3004', '3005', '3005',
        '3006', '3006', '3007', '3007', '3008',
        '3008', '3009', '3009', '3010', '3010',
        '3011', '3011', '3012', '3012', '3013',
        '3013', '3014', '3014', '3015', '3015',
        '3016', '3016', '3017', '3017', '3018',
        '3018', '3019', '3019', '3020', '3020'
    ],
    'text': [
        "People like you are ruining this community with your hate.",  # HS
        "Everyone deserves respect, regardless of their opinions.",  # CN
        "Your arguments are so backward and offensive.",  # HS
        "It's important to engage in discussions with empathy and understanding.",  # CN
        "Why do people like you always spread negativity?",  # HS
        "Constructive dialogue can bridge gaps and resolve conflicts.",  # CN
        "Your comment is harmful and discriminatory.",  # HS
        "Let's focus on positive and respectful communication.",  # CN
        "You should be ashamed of your prejudiced views.",  # HS
        "Everyone has different perspectives and that's okay.",  # CN
        "This kind of rhetoric only perpetuates hatred.",  # HS
        "Promoting mutual respect and understanding is crucial.",  # CN
        "Your behavior is unacceptable and damaging.",  # HS
        "We should strive for more inclusive and respectful conversations.",  # CN
        "Hate speech has no place in a healthy dialogue.",  # HS
        "Constructive criticism helps us grow, not tear us down.",  # CN
        "This kind of negativity only fosters division.",  # HS
        "Let's engage in discussions that promote empathy and learning.",  # CN
        "Your language is hurtful and unnecessary.",  # HS
        "Respectful dialogue is essential for productive conversations.",  # CN
        "People like you are spreading false information.",  # HS
        "We can have disagreements without resorting to personal attacks.",  # CN
        "Your comments are divisive and harmful.",  # HS
        "Understanding different viewpoints can lead to meaningful discussions.",  # CN
        "You only contribute to the problem with your hateful speech.",  # HS
        "Empathy and respect are the foundation of any healthy debate.",  # CN
        "This kind of discourse is unacceptable and damaging.",  # HS
        "Promoting kindness and open-mindedness can make a difference.",  # CN
        "Your views are toxic and unproductive.",  # HS
        "Let's work towards creating a more respectful and understanding environment.",  # CN
        "Your comments are inflammatory and disrespectful.",  # HS
        "Focusing on common ground can help us resolve conflicts.",  # CN
        "Your attitude is completely unacceptable.",  # HS
        "We should always strive for constructive and respectful conversations.",  # CN
        "Your statements are prejudiced and harmful.",  # HS
        "Encouraging respectful dialogue can lead to positive change.",  # CN
        "Your argument is completely unfounded and offensive.",  # HS
        "Let's focus on finding common ground rather than attacking each other.",  # CN
        "Your language is very hurtful and damaging.",  # HS
        "Promoting understanding is essential for constructive discussions.",  # CN
    ],
    'turn_id': [
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1
    ]
}

mock_dialogues_TikTok_df = pd.DataFrame(data)

# HS Prediction

## Functions

In [6]:
def preprocess_text(text):
    if isinstance(text, list):
        text = " ".join(text)
    elif not isinstance(text, str):
        print(f"Invalid text input: {text}")
        return ""
    
    # Lowercase the text
    text = text.lower()
    
    # Remove usernames, links, hashtags, and punctuation
    text = re.sub(r"@\w+", "", text)  # Remove usernames
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # Remove links
    text = re.sub(r"#(\w+)", r"\1", text)  # Remove hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    
    # Correct spelling
    # text = str(TextBlob(text).correct())
    
    # Remove any extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Preprocess comments dataset
def preprocess_comments(data):
    # Remove rows with missing text or duplicates
    data.dropna(subset=['text'], inplace=True)
    data = data.drop_duplicates(subset='text')
    data['preprocessed_text'] = [preprocess_text(text) for text in tqdm(data['text'], desc="Processing Texts")]
    
    return data

def tokenize_function(data):
    # Ensure that 'text' field is present and tokenize
    return tokenizer(data['text'], padding="max_length", truncation=True, max_length=128)

def predict_hate(dataset, model, tokenizer):
    # Create a DataLoader with batch size 16 and padding using the tokenizer
    loader = DataLoader(
        dataset, 
        batch_size=16, 
        collate_fn=DataCollatorWithPadding(tokenizer=tokenizer)
    )
    
    # Prepare lists to store texts and predictions
    texts, probabilities = [], []
    
    # Set the model to evaluation mode
    model.eval()
    
    with torch.no_grad():
        # Iterate over batches
        for batch in loader:
            # Get model outputs
            outputs = model(batch['input_ids'], attention_mask=batch['attention_mask'])
            # Convert logits to probabilities using softmax
            probs = torch.softmax(outputs.logits, dim=1)
            
            # Store text and probabilities
            for i in range(batch['input_ids'].size(0)):
                texts.append(tokenizer.decode(batch['input_ids'][i], skip_special_tokens=True))
                probabilities.append(probs[i].tolist())

    # Return results as a DataFrame
    return pd.DataFrame({
        'text': texts,
        'non_hate_probability': [p[0] for p in probabilities],  # Probability for non-hate speech
        'hate_probability': [p[1] for p in probabilities]   # Probability for hate speech
    })

## Training Model with labeled data from literature

In [ ]:
mock_data_hate_labeled = preprocess_comments(mock_data_hate_labeled)
mock_dataset_hate_labeled = Dataset.from_pandas(mock_data_hate_labeled)

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT", num_labels=2)


# Split the dataset into train/test/validation with StratifiedShuffleSplit
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42)
train_indices, temp_indices = next(splitter.split(mock_dataset_hate_labeled['label'], mock_dataset_hate_labeled['label']))
train_set = mock_dataset_hate_labeled.select(train_indices)
temp_set = mock_dataset_hate_labeled.select(temp_indices)

# Further split temp into validation and test sets
temp_labels = temp_set['label']
splitter_temp = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
validation_indices, test_indices = next(splitter_temp.split(temp_set['label'], temp_labels))
validation_set = temp_set.select(validation_indices)
test_set = temp_set.select(test_indices)


# Tokenize the datasets
tokenized_train = train_set.map(tokenize_function, batched=True)
tokenized_validation = validation_set.map(tokenize_function, batched=True)
tokenized_test = test_set.map(tokenize_function, batched=True)


In [ ]:


# Function to create and train a model with given hyperparameters
def train_model_with_params(learning_rate, batch_size, num_epochs):
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        eval_steps=500,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_validation,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model
    trainer.train()
    
    # Evaluate the model on the validation set
    eval_results = trainer.evaluate()
    eval_loss = eval_results["eval_loss"]
    
    return eval_loss

# Define hyperparameter grid as desired
hyperparameter_grid = {
    'learning_rate': [1e-5, 3e-5, 5e-5],
    'per_device_train_batch_size': [8, 16],
    'num_train_epochs': [3, 5]
}

# Grid search
best_params = None
best_loss = np.inf

for lr in hyperparameter_grid['learning_rate']:
    for batch_size in hyperparameter_grid['per_device_train_batch_size']:
        for num_epochs in hyperparameter_grid['num_train_epochs']:
            print(f"Training with lr={lr}, batch_size={batch_size}, num_epochs={num_epochs}")
            loss = train_model_with_params(lr, batch_size, num_epochs)
            print(f"Validation Evaluation Loss: {loss}")
            
            if loss < best_loss:
                best_loss = loss
                best_params = (lr, batch_size, num_epochs)

print(f"Best Hyperparameters: learning_rate={best_params[0]}, batch_size={best_params[1]}, num_epochs={best_params[2]}")

# Train final model with the best hyperparameters on the full training data
best_learning_rate, best_batch_size, best_num_epochs = best_params


In [ ]:
final_training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=best_learning_rate,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_num_epochs,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

final_trainer = Trainer(
    model=model,
    args=final_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
final_trainer.train()

# Save the model
model.save_pretrained("./SupervisedTrainingHateBERT")
tokenizer.save_pretrained("./SupervisedTrainingHateBERT")

# Evaluate on the test set
test_results = final_trainer.evaluate(eval_dataset=tokenized_test)
test_loss = test_results["eval_loss"]

print(f"Test Set Evaluation Loss: {test_loss}")


## Predicting TikTok messages

In [ ]:
# Load tokenizer and model for hate speech detection
tokenizer_hate = AutoTokenizer.from_pretrained("./SupervisedTrainingHateBERT")
model_hate = AutoModelForSequenceClassification.from_pretrained("./SupervisedTrainingHateBERT")

# Preprocess comments and get the preprocessed text
comments_data = preprocess_comments(mock_comments_df)
preprocessed_texts = comments_data['preprocessed_text'].to_list()

# Tokenize preprocessed texts using the hate speech tokenizer
encodings_hate = tokenizer_hate(preprocessed_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Create dataset for hate speech predictions
dataset_hate = Dataset.from_dict({
    'input_ids': encodings_hate['input_ids'],
    'attention_mask': encodings_hate['attention_mask']
})

# Make predictions
predictions_df = predict_hate(dataset_hate, model_hate, tokenizer_hate)

# You can then save the DataFrame to a CSV file if needed
predictions_file = "hateFirstTraining.csv"
predictions_df.to_csv(predictions_file, index=False)


## Obtain pseudolabels and retraining HS prediction model with labels and pseudo-labels

In [ ]:
# Define pseudolabels
hate_comments_predictions = pd.read_csv("hateFirstTraining.csv")

# Variables to control the pseudolabeling process
hate_probability_threshold_pseudolabel = False
non_hate_probability_threshold_pseudolabel = False

# Hate pseudolabeling based on probability threshold or quantile
if hate_probability_threshold_pseudolabel:
    probability_threshold = 0.95
    # Only keep rows with hate probability greater than the threshold
    hate_pseudolabels = hate_comments_predictions[hate_comments_predictions["hate_probability"] > probability_threshold]
    hate_pseudolabels["label"] = 1
else:
    quantile = 0.05
    # Get the top 5% of hate comments based on the hate probability
    top_5_percent_cutoff = hate_comments_predictions["hate_probability"].quantile(1 - quantile)
    hate_pseudolabels = hate_comments_predictions[hate_comments_predictions["hate_probability"] >= top_5_percent_cutoff]
    hate_pseudolabels["label"] = 1

# Non-hate pseudolabeling based on probability threshold or quantile
if non_hate_probability_threshold_pseudolabel:
    probability_threshold = 0.05
    # Only keep rows with hate probability less than the threshold
    non_hate_pseudolabels = hate_comments_predictions[hate_comments_predictions["hate_probability"] < probability_threshold]
    non_hate_pseudolabels["label"] = 0
else:
    quantile = 0.05
    # Get the bottom 5% of comments based on the hate probability
    bottom_5_percent_cutoff = hate_comments_predictions["hate_probability"].quantile(quantile)
    non_hate_pseudolabels = hate_comments_predictions[hate_comments_predictions["hate_probability"] <= bottom_5_percent_cutoff]
    non_hate_pseudolabels["label"] = 0

# Combine the hate and non-hate pseudolabels
hate_pseudolabels = pd.concat([hate_pseudolabels, non_hate_pseudolabels], ignore_index=True)

# Display the final pseudolabels dataset
hate_pseudolabels = hate_pseudolabels[["text", "label"]]
hate_pseudolabels

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT", num_labels=2)

# Merge the DataFrames
hate_dataset = pd.concat([mock_data_hate_labeled, hate_pseudolabels])

hate_dataset = Dataset.from_pandas(hate_dataset)

# Split the dataset into train/test/validation with StratifiedShuffleSplit
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, temp_indices = next(splitter.split(hate_dataset['label'], hate_dataset['label']))
train_set = hate_dataset.select(train_indices)
temp_set = hate_dataset.select(temp_indices)

# Further split temp into validation and test sets
temp_labels = temp_set['label']
splitter_temp = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
validation_indices, test_indices = next(splitter_temp.split(temp_set['label'], temp_labels))
validation_set = temp_set.select(validation_indices)
test_set = temp_set.select(test_indices)

# Tokenization function for dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

# Tokenize the datasets
tokenized_train = train_set.map(tokenize_function, batched=True)
tokenized_validation = validation_set.map(tokenize_function, batched=True)
tokenized_test = test_set.map(tokenize_function, batched=True)


In [ ]:
# Define hyperparameter grid
hyperparameter_grid = {
    'learning_rate': [1e-5, 3e-5, 5e-5],
    'per_device_train_batch_size': [8, 16],
    'num_train_epochs': [3, 5]
}

# Function to create and train a model with given hyperparameters
def train_and_evaluate(learning_rate, batch_size, num_epochs):
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        eval_steps=500,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_validation,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model
    trainer.train()
    
    # Evaluate on the test set
    test_results = trainer.evaluate(tokenized_test)
    eval_loss = test_results["eval_loss"]
    
    return eval_loss

# Grid search
best_params = None
best_loss = np.inf

for lr in hyperparameter_grid['learning_rate']:
    for batch_size in hyperparameter_grid['per_device_train_batch_size']:
        for num_epochs in hyperparameter_grid['num_train_epochs']:
            print(f"Training with lr={lr}, batch_size={batch_size}, num_epochs={num_epochs}")
            loss = train_and_evaluate(lr, batch_size, num_epochs)
            print(f"Evaluation Loss: {loss}")
            
            if loss < best_loss:
                best_loss = loss
                best_params = (lr, batch_size, num_epochs)

print(f"Best Parameters: learning_rate={best_params[0]}, batch_size={best_params[1]}, num_epochs={best_params[2]}")

# Use the best hyperparameters to train and save the final model
best_lr, best_batch_size, best_num_epochs = best_params


In [ ]:
final_training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_num_epochs,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

final_trainer = Trainer(
    model=model,
    args=final_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
final_trainer.train()

# Evaluate on the test set
test_results = final_trainer.evaluate(tokenized_test)
print("Test dataset results:", test_results)

# Save the final model and tokenizer
model.save_pretrained("./Semi_SupervisedTrainingHateBERT")
tokenizer.save_pretrained("./Semi_SupervisedTrainingHateBERT")

## Predicting final HS probabilities

In [ ]:
# Load resources
tokenizer_hate = AutoTokenizer.from_pretrained("./Semi_SupervisedTrainingHateBERT")
model_hate = AutoModelForSequenceClassification.from_pretrained("./Semi_SupervisedTrainingHateBERT")

# Preprocess comments and get the preprocessed text
comments_data = preprocess_comments(mock_comments_df)
preprocessed_texts = comments_data['preprocessed_text'].to_list()

# Tokenize preprocessed texts using the hate speech tokenizer
encodings_hate = tokenizer_hate(preprocessed_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Create dataset for hate speech predictions
dataset_hate = Dataset.from_dict({
    'input_ids': encodings_hate['input_ids'],
    'attention_mask': encodings_hate['attention_mask']
})

# Make predictions
predictions_df = predict_hate(dataset_hate, model_hate, tokenizer_hate)

# You can then save the DataFrame to a CSV file if needed
predictions_file = "finalHatePredictions.csv"
predictions_df.to_csv(predictions_file, index=False)
predictions_df

# CS Prediction

## Functions

In [22]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split

# Define the function to prepare data
def prepare_data(df):
    pairs, labels = [], []
    grouped = df.groupby('dialogue_id')
    for _, group in grouped:
        if len(group) % 2 != 0:
            group = group[:-1]
        
        for i in range(0, len(group), 2):
            combined_text = "[CLS] " + preprocess_text(group.iloc[i]['text']) + " [SEP] " + preprocess_text(group.iloc[i+1]['text'])
            combined_label = [group.iloc[i]['label'], group.iloc[i+1]['label']]
            pairs.append(combined_text)
            labels.append(combined_label)
    return pd.DataFrame({'text': pairs, 'labels': labels})

# Define the function to prepare data
def prepare_data_TikTok(df):
    pairs = []
    grouped = df.groupby('dialogue_id')
    for _, group in grouped:
        if len(group) % 2 != 0:
            group = group[:-1]
        
        for i in range(0, len(group), 2):
            combined_text = "[CLS] " + preprocess_text(group.iloc[i]['text']) + " [SEP] " + preprocess_text(group.iloc[i+1]['text'])
            pairs.append(combined_text)
    return pd.DataFrame({'text': pairs})
    
# Split data
def split_data(df):
    X = df['text']
    y = df['labels']
    
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    return X_train, X_validation, X_test, y_train, y_validation, y_test

# Define the Dataset class
class DialogueDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.texts = df['text'].tolist()
        self.labels = [torch.tensor(label, dtype=torch.float32) for label in df['labels']]
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': labels
        }

## Training Model with labeled data from literature

In [ ]:
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import pandas as pd

# Define a function to create and train a model with given hyperparameters
def train_model_with_params(learning_rate, batch_size, num_epochs):
    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        eval_steps=500,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False
    )

    # Initialize the model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    # Define the trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model
    trainer.train()
    
    # Evaluate the model on the validation set
    eval_results = trainer.evaluate()
    eval_loss = eval_results["eval_loss"]
    
    return eval_loss

# Prepare and split data
pairs_df = prepare_data(mock_dialogues_df)
X_train, X_validation, X_test, y_train, y_validation, y_test = split_data(pairs_df)

# Convert data to DataFrame
train_df = pd.DataFrame({'text': X_train, 'labels': y_train.tolist()})
validation_df = pd.DataFrame({'text': X_validation, 'labels': y_validation.tolist()})
test_df = pd.DataFrame({'text': X_test, 'labels': y_test.tolist()})

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create datasets
train_dataset = DialogueDataset(train_df, tokenizer)
validation_dataset = DialogueDataset(validation_df, tokenizer)
test_dataset = DialogueDataset(test_df, tokenizer)

# Define hyperparameter grid as desired
hyperparameter_grid = {
    'learning_rate': [1e-5, 3e-5, 5e-5],
    'per_device_train_batch_size': [8, 16],
    'num_train_epochs': [3, 5]
}

# Grid search
best_params = None
best_loss = np.inf

for lr in hyperparameter_grid['learning_rate']:
    for batch_size in hyperparameter_grid['per_device_train_batch_size']:
        for num_epochs in hyperparameter_grid['num_train_epochs']:
            print(f"Training with lr={lr}, batch_size={batch_size}, num_epochs={num_epochs}")
            loss = train_model_with_params(lr, batch_size, num_epochs)
            print(f"Validation Evaluation Loss: {loss}")
            
            if loss < best_loss:
                best_loss = loss
                best_params = (lr, batch_size, num_epochs)

print(f"Best Hyperparameters: learning_rate={best_params[0]}, batch_size={best_params[1]}, num_epochs={best_params[2]}")

# Retrain the model with the best hyperparameters on the full training data
best_learning_rate, best_batch_size, best_num_epochs = best_params


In [ ]:
# Define training arguments with best hyperparameters
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=best_learning_rate,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_num_epochs,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

# Initialize and train the final model with the best parameters
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)
trainer.train()

# Save the model and tokenizer
import os
os.makedirs("./SupervisedTrainingDialogueBERT", exist_ok=True)
model.save_pretrained("./SupervisedTrainingDialogueBERT")
tokenizer.save_pretrained("./SupervisedTrainingDialogueBERT")

# Evaluate on the test dataset
test_results = trainer.evaluate(test_dataset)
print("Test dataset results:", test_results)


## Predicting TikTok Dialogues

In [ ]:
# Load the model and tokenizer
tokenizer = BertTokenizer.from_pretrained("./SupervisedTrainingDialogueBERT")
model = BertForSequenceClassification.from_pretrained("./SupervisedTrainingDialogueBERT")

class PredictionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.texts = df['text'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten()
        }

pairs_df_TikTok = prepare_data_TikTok(mock_dialogues_TikTok_df)
# Create the dataset
prediction_dataset = PredictionDataset(pairs_df_TikTok, tokenizer)

# Create a dataloader
dataloader = DataLoader(prediction_dataset, batch_size=8)

def predict(dataloader, model):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
            
            all_preds.extend(probs)
    return all_preds

# Get predictions
probs = predict(dataloader, model)

pairs_df_TikTok["probabilities"] = probs
print("Probabilities:\n", probs)
print(len(probs))


## Obtain pseudolabels and retraining CS prediction model with labels and pseudo-labels

In [ ]:
pairs_df_TikTok[['probability_1', 'probability_2']] = pd.DataFrame(pairs_df_TikTok['probabilities'].tolist(), index=pairs_df_TikTok.index)

# Variables to control the pseudolabeling process
counter_speech_probability_threshold_pseudolabel = False
non_counter_speech_probability_threshold_pseudolabel = False

def assign_labels(df, threshold=None, quantile=None, is_counter_speech=True):
    labels = []
    for _, row in df.iterrows():
        prob = row[['probability_1', 'probability_2']].values
        max_prob = max(prob)
        index_max_prob = prob.argmax()
        
        if threshold is not None:
            if is_counter_speech:
                label = [0, 1] if prob[1] > threshold else [1, 0]
            else:
                label = [1, 0] if prob[1] < (1 - threshold) else [0, 1]
        else:
            if is_counter_speech:
                cutoff = pd.Series(prob).quantile(1 - quantile)
                label = [0, 1] if max_prob >= cutoff else [1, 0]
            else:
                cutoff = pd.Series(prob).quantile(quantile)
                label = [1, 0] if min(prob) <= cutoff else [0, 1]
        
        labels.append(label)
    return labels

# Pseudolabeling for counter speech
if counter_speech_probability_threshold_pseudolabel:
    probability_threshold = 0.95
    counter_speech_pseudolabels = pairs_df_TikTok[pairs_df_TikTok[['probability_1', 'probability_2']].max(axis=1) > probability_threshold]
    counter_speech_pseudolabels['labels'] = assign_labels(counter_speech_pseudolabels, threshold=probability_threshold, is_counter_speech=True)
else:
    quantile = 0.05
    top_5_percent_cutoff = pairs_df_TikTok[['probability_1', 'probability_2']].max(axis=1).quantile(1 - quantile)
    counter_speech_pseudolabels = pairs_df_TikTok[pairs_df_TikTok[['probability_1', 'probability_2']].max(axis=1) >= top_5_percent_cutoff]
    counter_speech_pseudolabels['labels'] = assign_labels(counter_speech_pseudolabels, quantile=quantile, is_counter_speech=True)

# Display the final DataFrame with pseudolabels
counter_speech_pseudolabels = counter_speech_pseudolabels[["text", "labels"]]
print(counter_speech_pseudolabels)

pairs_df_labeled = prepare_data(mock_dialogues_df)
pairs_df = pd.concat([pairs_df_labeled, counter_speech_pseudolabels])
pairs_df

In [ ]:
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import pandas as pd

# Define a function to create and train a model with given hyperparameters
def train_model_with_params(learning_rate, batch_size, num_epochs):
    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        eval_steps=500,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False
    )

    # Initialize the model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    # Define the trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model
    trainer.train()
    
    # Evaluate the model on the validation set
    eval_results = trainer.evaluate()
    eval_loss = eval_results["eval_loss"]
    
    return eval_loss

# Prepare and split data
pairs_df = prepare_data(mock_dialogues_df)
X_train, X_validation, X_test, y_train, y_validation, y_test = split_data(pairs_df)

# Convert data to DataFrame
train_df = pd.DataFrame({'text': X_train, 'labels': y_train.tolist()})
validation_df = pd.DataFrame({'text': X_validation, 'labels': y_validation.tolist()})
test_df = pd.DataFrame({'text': X_test, 'labels': y_test.tolist()})

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create datasets
train_dataset = DialogueDataset(train_df, tokenizer)
validation_dataset = DialogueDataset(validation_df, tokenizer)
test_dataset = DialogueDataset(test_df, tokenizer)

# Define hyperparameter grid as desired
hyperparameter_grid = {
    'learning_rate': [1e-5, 3e-5, 5e-5],
    'per_device_train_batch_size': [8, 16],
    'num_train_epochs': [3, 5]
}

# Grid search
best_params = None
best_loss = np.inf

for lr in hyperparameter_grid['learning_rate']:
    for batch_size in hyperparameter_grid['per_device_train_batch_size']:
        for num_epochs in hyperparameter_grid['num_train_epochs']:
            print(f"Training with lr={lr}, batch_size={batch_size}, num_epochs={num_epochs}")
            loss = train_model_with_params(lr, batch_size, num_epochs)
            print(f"Validation Evaluation Loss: {loss}")
            
            if loss < best_loss:
                best_loss = loss
                best_params = (lr, batch_size, num_epochs)

print(f"Best Hyperparameters: learning_rate={best_params[0]}, batch_size={best_params[1]}, num_epochs={best_params[2]}")

# Retrain the model with the best hyperparameters on the full training data
best_learning_rate, best_batch_size, best_num_epochs = best_params


In [ ]:
# Define training arguments with best hyperparameters
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=best_learning_rate,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_num_epochs,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

# Initialize and train the final model with the best parameters
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)
trainer.train()

# Save the model and tokenizer
import os
os.makedirs("./SemiSupervisedTrainingDialogueBERT", exist_ok=True)
model.save_pretrained("./SemiSupervisedTrainingDialogueBERT")
tokenizer.save_pretrained("./SemiSupervisedTrainingDialogueBERT")

# Evaluate on the test dataset
test_results = trainer.evaluate(test_dataset)
print("Test dataset results:", test_results)

## Predicting final CS probabilities

In [ ]:

# Load the model and tokenizer
tokenizer = BertTokenizer.from_pretrained("./SemiSupervisedTrainingDialogueBERT")
model = BertForSequenceClassification.from_pretrained("./SemiSupervisedTrainingDialogueBERT")

class PredictionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.texts = df['text'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten()
        }

pairs_df_TikTok = prepare_data_TikTok(mock_dialogues_TikTok_df)
# Create the dataset
prediction_dataset = PredictionDataset(pairs_df_TikTok, tokenizer)

# Create a dataloader
dataloader = DataLoader(prediction_dataset, batch_size=8)

def predict(dataloader, model):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
            
            all_preds.extend(probs)
    return all_preds

# Get predictions
probs = predict(dataloader, model)

pairs_df_TikTok["probabilities"] = probs
print("Probabilities:\n", probs)
print(len(probs))


# HS Comments Topic Modeling

## Functions

In [28]:
# Initialize tqdm pandas extension
def clean_text(text):
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Remove mentions, URLs, and hashtags
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'http\S+|www.\S+', '', text)  # Remove URLs
    text = re.sub(r'#', '', text)  # Remove the # symbol but keep the text following it

    # Define emoticon pattern
    emoticon_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F700-\U0001F77F"  # Alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric shapes extended
        u"\U0001F800-\U0001F8FF"  # Supplemental arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental symbols and pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and pictographs extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"  # Enclosed characters
        "]+", flags=re.UNICODE)
    
    # Remove emoticons
    text = emoticon_pattern.sub(r'', text)
    
    # Remove punctuation
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text.strip()


def calculate_coherence_score(topic_model, docs, dictionary):
    topics = topic_model.get_topics()

    topic_list = [[word for word, _ in topic] for topic in topics.values()]
    
    texts = [doc.split() for doc in docs]
    
    dictionary = Dictionary(texts)

    coherence_model = CoherenceModel(topics=topic_list, texts=texts, dictionary=dictionary, coherence='c_v')
    return coherence_model.get_coherence()

## Main Code

In [29]:
commentsData = pd.read_csv("finalHatePredictions.csv")
commentsData["is_hate_speech"] = (commentsData["hate_probability"] > 0.5).astype(int)
hateMessages = commentsData[commentsData['is_hate_speech'] == 1]
hateMessages['clean_text'] = hateMessages['text'].apply(lambda x: clean_text(x))
hateMessages = hateMessages[hateMessages['clean_text'].str.strip().astype(bool)]

hate_speech_docs = hateMessages['clean_text'].tolist()
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(hate_speech_docs)

In [ ]:

representation_model = {
    "KeyBERT": KeyBERTInspired(),
    "POS": PartOfSpeech("en_core_web_trf"),
    "KeyBERT_MMR": [KeyBERTInspired(top_n_words=10), MaximalMarginalRelevance(diversity=.8)],
}

# Define a range of parameters to test
vectorizer_params = [
    {'stop_words': 'english', 'ngram_range': (1, 1)},
    # Add more parameter configurations here
]

hdbscan_params = [
    {'min_cluster_size': 2, 'metric': 'euclidean', 'cluster_selection_method': 'eom', 'prediction_data': True},
    {'min_cluster_size': 2, 'metric': 'manhattan', 'cluster_selection_method': 'leaf', 'prediction_data': True},
    # Add more parameter configurations here
]

best_coherence = -float('inf')
best_params = {}

# Convert documents to dictionary format for coherence calculation
dictionary = Dictionary([doc.split() for doc in hate_speech_docs])

for vectorizer_param in vectorizer_params:
    for hdbscan_param in hdbscan_params:
        vectorizer_model = CountVectorizer(**vectorizer_param)
        hdbscan_model = hdbscan.HDBSCAN(**hdbscan_param)

        topic_model = BERTopic(
            language="english",
            embedding_model=embedding_model,
            vectorizer_model=vectorizer_model,
            representation_model=representation_model,
            verbose=True,
            calculate_probabilities=True,
            hdbscan_model=hdbscan_model
        )
        
        topics, probs = topic_model.fit_transform(hate_speech_docs, embeddings)
        
        coherence_score = calculate_coherence_score(topic_model, hate_speech_docs, dictionary)
        
        if coherence_score > best_coherence:
            best_coherence = coherence_score
            best_params = {
                'vectorizer': vectorizer_param,
                'hdbscan': hdbscan_param
            }

# Print the best parameters and their coherence score
print("Best Coherence Score:", best_coherence)
print("Best Parameters:", best_params)

# Final model with best parameters
best_vectorizer_model = CountVectorizer(**best_params['vectorizer'])
best_hdbscan_model = hdbscan.HDBSCAN(**best_params['hdbscan'])

final_topic_model = BERTopic(
    language="english",
    embedding_model=embedding_model,
    vectorizer_model=best_vectorizer_model,
    representation_model=representation_model,
    top_n_words=10,
    verbose=True,
    calculate_probabilities=True,
    hdbscan_model=best_hdbscan_model
)

# Assuming hate_speech_docs and embeddings are already defined
topics, probs = final_topic_model.fit_transform(hate_speech_docs, embeddings)
hateMessages['Topic'] = topics
topic_info = final_topic_model.get_topic_info()

hateMessagesWithTopicInfo = pd.merge(
    hateMessages, 
    topic_info[['Topic', 'Representation', 'KeyBERT', 'POS', 'KeyBERT_MMR', 'Representative_Docs']], 
    on='Topic', 
    how='left'
)

hateMessagesWithTopicInfo.to_csv('hateMessagesWithTopicInformation.csv', index=False)
hateMessagesWithTopicInfo


In [ ]:
# Save the topic_model
with open('topic_model_hate_comments.pkl', 'wb') as model_file:
    pickle.dump(topic_model, model_file)

# Save the topics
with open('topics_hate_comments.pkl', 'wb') as topics_file:
    pickle.dump(topics, topics_file)

# Save the probabilities
with open('probs_hate_comments.pkl', 'wb') as probs_file:
    pickle.dump(probs, probs_file)

print("Topic model, topics, and probabilities have been saved.")

## Labeling Topics

In [ ]:
# Load the topic_model
with open('topic_model_hate_comments.pkl', 'rb') as model_file:
    topic_model = pickle.load(model_file)

# Load the topics
with open('topics_hate_comments.pkl', 'rb') as topics_file:
    topics = pickle.load(topics_file)

# Load the probabilities
with open('probs_hate_comments.pkl', 'rb') as probs_file:
    probs = pickle.load(probs_file)

print("Topic model, topics, and probabilities have been loaded.")

In [ ]:
topic_info = topic_model.get_topic_info()

# This step requires having the LLM downloaded
filename_llm = "openhermes-2.5-mistral-7b.Q4_K_M.gguf"
llm = Llama(
    model_path=filename_llm,
    n_gpu_layers=-1,
    n_ctx=4096,
    stop=["Q:", "\n"]
)

def generate_label_for_topic(row):
    combined_words = []
    seen_words = set()

    for source in ['KeyBERT', 'POS', 'KeyBERT_MMR']:
        for word in row[source]:
            if word not in seen_words:
                seen_words.add(word)
                combined_words.append(word)

    combined_description = " ".join(combined_words)

    prompt = f"These are the top words of a topic resulting from topic modeling: {top_words}. Provide a concise label for this topic, limited to a maximum of 5 words:"
    response = llm(prompt)

    try:
        print("Response: ", response)
        label_text = response['choices'][0]['text'].strip()
        return label_text
    except (KeyError, IndexError, TypeError) as e:
        print("Failed to extract label due to:", str(e))
        return "Label extraction failed"

tqdm.pandas(desc="Generating Labels")
topic_info['CustomName'] = topic_info.progress_apply(generate_label_for_topic, axis=1)

# Extract the topic labels for all topics
topic_labels = topic_info.set_index('Topic')['CustomName'].to_dict()

# Ensure all labels are strings and filter out "No Label"
filtered_topic_labels = {k: v for k, v in topic_labels.items() if pd.notna(v) and v != "Label extraction failed"}

In [ ]:
topic_info.to_csv("topicInformation_hate_comments.csv")
topic_info